In [ ]:
cd ..

In [ ]:
import torch.optim as optim
import torch.nn as nn

from tree_to_sequence.program_datasets import ForLambdaDataset
from tree_to_sequence.tree_to_sequence import TreeToSequence
from tree_to_sequence.tree_to_sequence_attention import TreeToSequenceAttention
from tree_to_sequence.tree_encoder import TreeEncoder
from tree_to_sequence.sequence_encoder import SequenceEncoder
from tree_to_sequence.multilayer_lstm_cell import MultilayerLSTMCell
from neural_nets_library import training

In [ ]:
num_vars = 10
num_ints = 11

for_ops = {
    "Var": 0,
    "Const": 1,
    "Plus": 2,
    "Minus": 3,
    "EqualFor": 4,
    "LeFor": 5,
    "GeFor": 6,
    "Assign": 7,
    "If": 8,
    "Seq": 9,
    "For": 10
}

for_ops = {"<" + k.upper() + ">": v for k,v in for_ops.items()}

lambda_ops = {
    "Var": 0,
    "Const": 1,
    "Plus": 2,
    "Minus": 3,
    "EqualFor": 4,
    "LeFor": 5,
    "GeFor": 6,
    "If": 7,
    "Let": 8,
    "Unit": 9,
    "Letrec": 10,
    "App": 11
}

lambda_ops = {"<" + k.upper() + ">": v for k,v in lambda_ops.items()}

lambda_calculus_ops = {
                "<VARIABLE>": 0,
                "<ABSTRACTION>": 1,
                "<NUMBER>": 2,
                "<BOOLEAN>": 3,
                "<NIL>": 4,
                "<IF>": 5,
                "<CONS>": 6,
                "<MATCH>": 7,
                "<UNARYOPER>": 8,
                "<BINARYOPER>": 9,
                "<LET>": 10,
                "<LETREC>": 11,
                "<TRUE>": 12,
                "<FALSE>": 13,
                "<TINT>": 14,
                "<TBOOL>": 15,
                "<TINTLIST>": 16,
                "<TFUN>": 17,
                "<ARGUMENT>": 18,
                "<NEG>": 19,
                "<NOT>": 20,
                "<PLUS>": 21,
                "<MINUS>": 22,
                "<TIMES>": 23,
                "<DIVIDE>": 24,
                "<AND>": 25,
                "<OR>": 26,
                "<EQUAL>": 27,
                "<LESS>": 28,
                "<APPLICATION>": 29,
                "<HEAD>": 30,
                "<TAIL>": 31
            }

In [ ]:
input_as_seq = False

use_embedding = True
binarize = False
eos_tokens = False

eos_bonus = 1 if eos_tokens else 0
long_base_case = True


In [ ]:
for_lambda_dset = ForLambdaDataset("ANC/Easy-arbitraryForList.json", binarize=binarize, 
                                   eos_tokens=eos_tokens, use_embedding=use_embedding, 
                                   long_base_case=long_base_case, input_as_seq=input_as_seq)

In [ ]:
def reset_all_parameters_uniform(model, stdev):
    for param in model.parameters():
        nn.init.uniform(param, -stdev, stdev)

In [ ]:
embedding_size = 256
hidden_size = 256
nclass = num_vars + num_ints + len(lambda_ops.keys())
num_layers = 1
attention = True
alignment_size = 50
align_type = 1
encoder_input_size = num_vars + num_ints + len(for_ops.keys()) + eos_bonus

if input_as_seq:
    encoder = SequenceEncoder(encoder_input_size, hidden_size, num_layers, attention=attention, use_embedding=use_embedding)
else:
    encoder = TreeEncoder(encoder_input_size, hidden_size, num_layers, [1, 2], attention=attention, use_embedding=use_embedding)
    
if attention:
    decoder = MultilayerLSTMCell(embedding_size + hidden_size, hidden_size, num_layers)
    program_model = TreeToSequenceAttention(encoder, decoder, hidden_size, nclass, embedding_size, alignment_size=alignment_size, align_type=align_type)
else:
    decoder = MultilayerLSTMCell(embedding_size, hidden_size, num_layers)
    program_model = TreeToSequence(encoder, decoder, hidden_size, nclass, embedding_size)
    
reset_all_parameters_uniform(program_model, 0.1)
encoder.initialize_forget_bias(3)
decoder.initialize_forget_bias(3)

In [ ]:
# program_model = program_model.cuda()

In [ ]:
def program_accuracy(prediction, target):
    return 1 if list(target.data) == prediction else 0

def token_accuracy(prediction, target):
    pass

optimizer = optim.Adam(program_model.parameters(), lr=0.005)
lr_scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, verbose=True, patience=100, factor=0.8)

In [ ]:
model, train_loss, validation_loss = training.train_model_anc(program_model, for_lambda_dset, optimizer,
                             lr_scheduler=lr_scheduler,
                             num_epochs=10, validation_criterion=program_accuracy, batch_size=100,
                             use_cuda=False, plateau_lr=True)

In [ ]:
import importlib
# importlib.reload(MultilayerLSTMCell)
print(for_lambda_dset[0][1])